In [16]:
using IntervalArithmetic
using LaTeXStrings
using Plots

In [17]:
include("./method4/index.jl")
include("./intervalPCM/index.jl")
include("./twofoldInterval/index.jl")
include("./twofoldIntervalPCM/index.jl")

print()

In [18]:
function laTeXMatrix(
        pattern,
        size::Tuple{Integer, Integer})::String
    m, n = size

    mat = "\\begin{bmatrix}"
    for i = 1:m, j = 1:n
        mat *= pattern(i, j)
        if j == n
            if i != m
                mat *= " \\\\ "
            end
        else
            mat *= " & "
        end
    end
    mat *= "\\end{bmatrix}"

    return mat
end

laTeXMatrix (generic function with 1 method)

In [19]:
function method4(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    _, n = size(A₁)

    results = solveApproximationLP_m4.([A₁, A₂])
    A = generateTwofoldIntervalMatrix_m4(results)
    println("A = $(twofoldIntervalMatrixLaTeXString(A))"); println()

    result = solveCancellingLP_m4(A)
    resultₛₜᵣ = cancellingLPResultLaTeXString_m4(result)
    println("W = $(resultₛₜᵣ.W)"); println()
    println("""
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{L}-}", (n, 1))) = $(resultₛₜᵣ.wᴸ⁻), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{U}-}", (n, 1))) = $(resultₛₜᵣ.wᵁ⁻), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{L}+}", (n, 1))) = $(resultₛₜᵣ.wᴸ⁺), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{U}+}", (n, 1))) = $(resultₛₜᵣ.wᵁ⁺)
        """); println()
    println("""
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^\\text{L}", (n, 1))) = $(resultₛₜᵣ.εᴸ), ~~
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^\\text{U}", (n, 1))) = $(resultₛₜᵣ.εᵁ)
        """); println()

    Â = updatePCM_m4(A, result)

    displayCancellingLPResultLaTeXString_m4(result)
    display(L"A = %$(twofoldIntervalMatrixLaTeXString(Â))")

    return
end

method4 (generic function with 1 method)

# example 1

In [20]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
A₂ = A₁

method4(A₁, A₂)

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   1.0, \left[   1.0,   2.0 \right],   2.0 \right] & \left[ 0.667, \left[ 0.667, 1.333 \right], 1.333 \right] & \left[   1.0, \left[   1.0, 1.333 \right], 1.333 \right] \\ \left[   0.5, \left[   0.5,   1.0 \right],   1.0 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 0.444, \left[ 0.444,   1.0 \right],   1.0 \right] & \left[ 0.667, \left[ 0.667,   1.0 \right],   1.0 \right] \\ \left[  0.75, \left[  0.75,   1.5 \right],   1.5 \right] & \left[   1.0, \left[   1.0,  2.25 \right],  2.25 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   1.0, \left[   1.0,   1.5 \right],   1.5 \right] \\ \left[  0.75, \left[  0.75,   1.0 \right],   1.0 \right] & \left[   1.0, \left[   1.0,   1.5 \right],   1.5 \right] & \left[ 0.667, \left[ 0.667,   1.0 \right],   1.0 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

W = \begin{bmatrix}\

L"$W = \begin{bmatrix}\left[ 0.235, 0.314 \right] \\ \left[ 0.157, 0.235 \right] \\ \left[ 0.235, 0.353 \right] \\ \left[ 0.235, 0.235 \right]\end{bmatrix}$"

L"$w^{\text{L}-} = \begin{bmatrix} 0.235  \\  0.157  \\  0.235  \\  0.235 \end{bmatrix}, ~~
w^{\text{U}-} = \begin{bmatrix} 0.314  \\  0.235  \\  0.353  \\  0.235 \end{bmatrix}, ~~
w^{\text{L}+} = \begin{bmatrix} 0.235  \\  0.157  \\  0.235  \\  0.235 \end{bmatrix}, ~~
w^{\text{U}+} = \begin{bmatrix} 0.314  \\  0.235  \\  0.353  \\  0.235 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}
$"

"\$A = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[   1.0, \\left[   1.0,   2.0 \\right],   2.0 \\right] & \\left[ 0.667, \\left[ 0.667, 1.333 \\right], 1.333 \\right] & \\left[   1.0, \\left[   1.0, 1.333 \\right], 1.333 \\right] \\\\ \\left[   0.5," ⋯ 439 bytes ⋯ "ht],   1.5 \\right] \\\\ \\left[  0.75, \\left[  0.75,   1.0 \\right],   1.0 \\right] & \\left[   1.0, \\left[   1.0,   1.5 \\right],   1.5 \\right] & \\left[ 0.667, \\left[ 0.667,   1.0 \\right],   1.0 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"



\begin{bmatrix}w_1^{\text{L}-} \\ w_2^{\text{L}-} \\ w_3^{\text{L}-} \\ w_4^{\text{L}-}\end{bmatrix} = \begin{bmatrix} 0.235  \\  0.157  \\  0.235  \\  0.235 \end{bmatrix}, ~~
\begin{bmatrix}w_1^{\text{U}-} \\ w_2^{\text{U}-} \\ w_3^{\text{U}-} \\ w_4^{\text{U}-}\end{bmatrix} = \begin{bmatrix} 0.314  \\  0.235  \\  0.353  \\  0.235 \end{bmatrix}, ~~
\begin{bmatrix}w_1^{\text{L}+} \\ w_2^{\text{L}+} \\ w_3^{\text{L}+} \\ w_4^{\text{L}+}\end{bmatrix} = \begin{bmatrix} 0.235  \\  0.157  \\  0.235  \\  0.235 \end{bmatrix}, ~~
\begin{bmatrix}w_1^{\text{U}+} \\ w_2^{\text{U}+} \\ w_3^{\text{U}+} \\ w_4^{\text{U}+}\end{bmatrix} = \begin{bmatrix} 0.314  \\  0.235  \\  0.353  \\  0.235 \end{bmatrix}


\begin{bmatrix}\varepsilon_1^\text{L} \\ \varepsilon_2^\text{L} \\ \varepsilon_3^\text{L} \\ \varepsilon_4^\text{L}\end{bmatrix} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}, ~~
\begin{bmatrix}\varepsilon_1^\text{U} \\ \varepsilon_2^\text{U} \\ \varepsilon_3^\text{U} \\ \vareps

# example 2

In [6]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
_, n = size(A₁)
A₂ = fill(1..1, (n, n))
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 4 / 5)..(A₁[i,j].hi * 5 / 4)
end

method4(A₁, A₂)

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   0.8, \emptyset,   2.5 \right] & \left[ 0.533, \emptyset, 1.667 \right] & \left[   0.8, \emptyset, 1.667 \right] \\ \left[   0.4, \emptyset,  1.25 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 0.356, \emptyset,  1.25 \right] & \left[ 0.533, \left[ 0.667, 0.667 \right],  1.25 \right] \\ \left[   0.6, \emptyset, 1.875 \right] & \left[   0.8, \emptyset, 2.812 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   0.8, \emptyset, 1.875 \right] \\ \left[   0.6, \emptyset,  1.25 \right] & \left[   0.8, \left[   1.5,   1.5 \right], 1.875 \right] & \left[ 0.533, \emptyset,  1.25 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

W = \begin{bmatrix}\left[ 0.22, 0.22 \right] \\ \left[ 0.147, 0.147 \right] \\ \left[ 0.413, 0.413 \right] \\ \left[ 0.22, 0.22 \right]\end{bmatrix}

\begin{bmatrix}w_1^{\text{L}-} \\ w_2^{\text{L}-} 

<!-- ともに弱い整合性を持ち， $A_{1ij} \subseteq A_{2ij}$

```
A₁ = [
    1..1 2..5 2..4 2..3
    (1/5)..(1/2) 1..1 (1/5)..4 (1/5)..3
    (1/4)..(1/2) (1/4)..5 1..1 (1/4)..3
    (1/3)..(1/2) (1/3)..5 (1/3)..4 1..1
]
A₂ = fill(1..1, size(A₁))
_, n = size(A₁)
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 1/1.1)..(A₁[i,j].hi * 1.1)
end

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

display(isConsistentIntervalPCM(A₁))
display(isWeaklyConsistentIntervalPCM(A₁))
display(isConsistentIntervalPCM(A₂))
display(isWeaklyConsistentIntervalPCM(A₂))

plotAll(A₁, A₂)
``` -->

# example 3

In [7]:
A = [
    1 2 4 4
    1/2 1 2 2
    1/4 1/2 1 1
    1/4 1/2 1 1
]

A₁ = randamizedIntervalPCM(A, 1234, 3e-2)
A₂ = randamizedIntervalPCM(A, 134, 3e-2)

method4(A₁, A₂)

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 1.981, \emptyset, 2.033 \right] & \left[ 3.944, \emptyset, 4.109 \right] & \left[ 3.937, \emptyset, 4.061 \right] \\ \left[ 0.492, \emptyset, 0.505 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 1.944, \emptyset, 2.075 \right] & \left[ 1.942, \emptyset, 2.045 \right] \\ \left[ 0.243, \emptyset, 0.254 \right] & \left[ 0.482, \emptyset, 0.515 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 0.963, \emptyset,  1.03 \right] \\ \left[ 0.246, \emptyset, 0.254 \right] & \left[ 0.489, \emptyset, 0.515 \right] & \left[ 0.971, \emptyset, 1.038 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

W = \begin{bmatrix}\left[ 0.503, 0.503 \right] \\ \left[ 0.247, 0.247 \right] \\ \left[ 0.122, 0.122 \right] \\ \left[ 0.127, 0.127 \right]\end{bmatrix}

\begin{bmatrix}w_1^{\text{L}-} \\ w_2^{\text{L}-} \\ w_3^{\text{L}-} \\ w_4^{\text

# example 4

In [8]:
A = [
    1 2 3 10
    1/2 1 3 8
    1/3 1/3 1 3
    1/10 1/8 1/3 1
]

A₁ = randamizedIntervalPCM(A, 124, 3.5e-1)
A₂ = randamizedIntervalPCM(A, 4, 3.5e-1)

method4(A₁, A₂)

A = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 1.477, \emptyset, 2.772 \right] & \left[ 2.334, \emptyset, 5.757 \right] & \left[ 8.353, \emptyset, 15.515 \right] \\ \left[ 0.361, \emptyset, 0.677 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 0.864, \emptyset, 3.835 \right] & \left[ 3.163, \emptyset, 10.334 \right] \\ \left[ 0.174, \emptyset, 0.428 \right] & \left[ 0.261, \emptyset, 1.157 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 1.523, \emptyset, 6.475 \right] \\ \left[ 0.064, \emptyset,  0.12 \right] & \left[ 0.097, \emptyset, 0.316 \right] & \left[ 0.154, \emptyset, 0.656 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

W = \begin{bmatrix}\left[ 0.625, 0.625 \right] \\ \left[ 0.226, 0.226 \right] \\ \left[ 0.109, 0.109 \right] \\ \left[ 0.04, 0.04 \right]\end{bmatrix}

\begin{bmatrix}w_1^{\text{L}-} \\ w_2^{\text{L}-} \\ w_3^{\text{L}-} \\ w_4^{\text